In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import library
import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

from ds_utils.clustering import vectorize, mbkmeans_clusters
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

nltk.download("stopwords")

SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv("./drive/MyDrive/Semester8/STBI/wiki_movie_plots_deduped.csv", index_col=0)


In [ ]:
df.info

<bound method DataFrame.info of                                          Title Origin/Ethnicity  \
Release Year                                                      
1901                    Kansas Saloon Smashers         American   
1901             Love by the Light of the Moon         American   
1901                   The Martyred Presidents         American   
1901          Terrible Teddy, the Grizzly King         American   
1902                    Jack and the Beanstalk         American   
...                                        ...              ...   
2014                         The Water Diviner          Turkish   
2017                        Çalgı Çengi İkimiz          Turkish   
2017                              Olanlar Oldu          Turkish   
2017                          Non-Transferable          Turkish   
2017                        İstanbul Kırmızısı          Turkish   

                                        Director  \
Release Year                                

In [ ]:
df.describe(include=object)

,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
count,34886,34886,34886,33464,34886,34886,34886
unique,32432,24,12593,32182,2265,34070,33869
top,Cinderella,American,Unknown,Tom and Jerry,unknown,https://en.wikipedia.org/wiki/Digimon_Adventur...,"(マッスル人参争奪！超人大戦争, Massuru Ninjin Soudatsu! Chou..."
freq,8,17377,1124,80,6083,5,6


In [ ]:
df.isna().sum()

Title                  0
Origin/Ethnicity       0
Director               0
Cast                1422
Genre                  0
Wiki Page              0
Plot                   0
dtype: int64

In [ ]:
result = df.drop(['Origin/Ethnicity', 'Director', 'Cast', 'Wiki Page', 'Genre'], axis = 1)

In [ ]:
result

,Title,Plot
Release Year,,
1901,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr..."
1901,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov..."
1901,The Martyred Presidents,"The film, just over a minute long, is composed..."
1901,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...
1902,Jack and the Beanstalk,The earliest known adaptation of the classic f...
...,...,...
2014,The Water Diviner,"The film begins in 1919, just after World War ..."
2017,Çalgı Çengi İkimiz,"Two musicians, Salih and Gürkan, described the..."
2017,Olanlar Oldu,"Zafer, a sailor living with his mother Döndü i..."


In [ ]:
print(str(result.describe(include=object)))

             Title                                               Plot
count        34886                                              34886
unique       32432                                              33869
top     Cinderella  (マッスル人参争奪！超人大戦争, Massuru Ninjin Soudatsu! Chou...
freq             8                                                  6


In [ ]:
result =result.drop_duplicates('Plot')
result.describe()

,Title,Plot
count,33869,33869
unique,31977,33869
top,Cinderella,"A bartender is working at a saloon, serving dr..."
freq,7,1


In [ ]:
# remove punctuation
def punctuation(txt):
  return re.sub(r"[^\w\s]","", str(txt))

result['Plot'] = result['Plot'].apply(punctuation)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
import nltk
nltk.download('punkt')

# tokenization
def word_tokenize_wrapper(text):
  return word_tokenize(text)
result = result['Plot'].apply(word_tokenize_wrapper)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#stopword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)
def stopword(text):
  words = [word for word in text if word.lower() not in sw_nltk]
  return words
result = result.apply(stopword)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
result

Release Year
1901    [bartender, working, saloon, serving, drinks, ...
1901    [moon, painted, smiling, face, hangs, park, ni...
1901    [film, minute, long, composed, two, shots, fir...
1901    [Lasting, 61, seconds, consisting, two, shots,...
1902    [earliest, known, adaptation, classic, fairyta...
                              ...                        
2014    [film, follows, elevenyearold, boy, named, Asl...
2017    [Two, musicians, Salih, Gürkan, described, adv...
2017    [Zafer, sailor, living, mother, Döndü, coastal...
2017    [film, centres, around, young, woman, named, A...
2017    [writer, Orhan, Şahin, returns, İstanbul, many...
Name: Plot, Length: 33869, dtype: object

In [ ]:
#normalization
nltk.download('omw-1.4')

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
def lemma(text):
  lemmatizer = WordNetLemmatizer()
  Output= [lemmatizer.lemmatize(words_sent) for words_sent in text]
  return Output
result = result.apply(lemma)


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
result

Release Year
1901    [bartender, working, saloon, serving, drink, c...
1901    [moon, painted, smiling, face, hang, park, nig...
1901    [film, minute, long, composed, two, shot, firs...
1901    [Lasting, 61, second, consisting, two, shot, f...
1902    [earliest, known, adaptation, classic, fairyta...
                              ...                        
2014    [film, follows, elevenyearold, boy, named, Asl...
2017    [Two, musician, Salih, Gürkan, described, adve...
2017    [Zafer, sailor, living, mother, Döndü, coastal...
2017    [film, centre, around, young, woman, named, Am...
2017    [writer, Orhan, Şahin, return, İstanbul, many,...
Name: Plot, Length: 33869, dtype: object

In [ ]:
# vocabulary
docs = result.values
tokenized_docs = result.values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

len(vocab)

193335

In [ ]:
vocab.most_common(10)

[('find', 29505),
 ('get', 28664),
 ('go', 26262),
 ('take', 25868),
 ('tell', 22728),
 ('one', 22592),
 ('father', 21631),
 ('love', 21171),
 ('life', 20679),
 ('back', 20284)]

In [ ]:
#word embedding with word2vec
model1 = Word2Vec(sentences=tokenized_docs, size = 100, workers=1, seed=42)

In [ ]:
# test model
model1.wv.most_similar("love", topn=5)

[('feeling', 0.6727650165557861),
 ('disrepair', 0.6019713878631592),
 ('headoverheels', 0.5951131582260132),
 ('Suspicion', 0.5648916959762573),
 ('romance', 0.5644127130508423)]

In [ ]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=50, print_silhouette_values=True)
df_clusters = pd.DataFrame({
    "text": docs,
    "tokens": [" ".join(text) for text in tokenized_docs],
    "cluster": cluster_labels
})

In [ ]:
print("Top terms per cluster (based on centroids):")
for i in range(50):
    tokens_per_cluster = ""
    most_representative = model1.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

In [ ]:
test_cluster = 48
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:20]:
    print(docs[d])
    print("-------------")

In [ ]:
vectorized_docs = vectorize(df, model = model1, strategy="average")
len(vectorized_docs), len(vectorized_docs[0])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(data):
    tfidf = TfidfVectorizer( stop_words='english',use_idf=True)
    tfidf_matrix = tfidf.fit_transform(data)
    return tfidf_matrix

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Generate matrix of word vectors
tfidf_matrix = vectorizer.fit_transform(df['Plot'])

# Print the shape of tfidf_matrix
print(tfidf_matrix.shape)

(34886, 133863)


In [ ]:
# in order to explore which documents have more similar respresentaiton, consine simliartiy can be used
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

In [ ]:
from collections import Counter

# vocabulary
docs = df.values
tokenized_docs = df.values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

len(vocab)

147112

In [ ]:
vectorizer3 = TfidfVectorizer(stop_words = stop_words, tokenizer = tokenize, max_features = 1000)
X3 = vectorizer3.fit_transform(desc)
words = vectorizer3.get_feature_names()

NameError: ignored

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0)
    kmeans.fit(X3)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.savefig('elbow.png')
plt.show()

NameError: ignored

In [ ]:
#word embedding with word2vec
from gensim.models import Word2Vec

model1 = Word2Vec(sentences=result, workers=1, seed=42)

In [ ]:
from ds_utils.clustering import vectorize, mbkmeans_clusters

vectorized_docs = vectorize(df, model = model1, strategy="average")
len(vectorized_docs), len(vectorized_docs[0])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(7, 100)

In [ ]:
from ds_utils.clustering import vectorize, mbkmeans_clusters
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=7, print_silhouette_values=True)
df_clusters = pd.DataFrame({
    "text": docs,
    "tokens": [" ".join(text) for text in tokenized_docs],
    "cluster": cluster_labels
})

For n_clusters = 7
Silhouette coefficient: 0.29
Inertia:0.31265548966590145
Silhouette values:


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


ValueError: ignored

In [ ]:
df.iloc[0]['Plot']